In [10]:
import concurrent.futures
import pandas as pd 
import numpy as np 
import spacy
import os 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 

In [11]:
os.getcwd()

'/Users/nikhilkudupudi/Projects - School & Work/NLP/Project'

In [12]:
files_list = os.listdir('resumes_corpus')
files_list.sort()
files_list

['00001.lab',
 '00001.txt',
 '00002.lab',
 '00002.txt',
 '00003.lab',
 '00003.txt',
 '00004.lab',
 '00004.txt',
 '00005.lab',
 '00005.txt',
 '00006.lab',
 '00006.txt',
 '00007.lab',
 '00007.txt',
 '00008.lab',
 '00008.txt',
 '00009.lab',
 '00009.txt',
 '00010.lab',
 '00010.txt',
 '00011.lab',
 '00011.txt',
 '00012.lab',
 '00012.txt',
 '00013.lab',
 '00013.txt',
 '00014.lab',
 '00014.txt',
 '00015.lab',
 '00015.txt',
 '00016.lab',
 '00016.txt',
 '00017.lab',
 '00017.txt',
 '00018.lab',
 '00018.txt',
 '00019.lab',
 '00019.txt',
 '00020.lab',
 '00020.txt',
 '00021.lab',
 '00021.txt',
 '00022.lab',
 '00022.txt',
 '00023.lab',
 '00023.txt',
 '00024.lab',
 '00024.txt',
 '00025.lab',
 '00025.txt',
 '00026.lab',
 '00026.txt',
 '00027.lab',
 '00027.txt',
 '00028.lab',
 '00028.txt',
 '00029.lab',
 '00029.txt',
 '00030.lab',
 '00030.txt',
 '00031.lab',
 '00031.txt',
 '00032.lab',
 '00032.txt',
 '00033.lab',
 '00033.txt',
 '00034.lab',
 '00034.txt',
 '00035.lab',
 '00035.txt',
 '00036.lab',
 '0003

In [13]:
import os
import numpy as np

file_contents = {'txt': [], 'lab': []}

for file in files_list:

    with open(f'resumes_corpus/{file}', 'r', encoding='utf-8', errors='ignore') as infile:
        content = infile.read()

        if file[-3:] == 'lab':
            file_contents['lab'].append(content)
            
        else:
            file_contents['txt'].append(content)


In [14]:
resume_df = pd.DataFrame(file_contents)
resume_df 

,txt,lab
0,"Database Administrator <span class=""hl"">Databa...",Database_Administrator
1,"Database Administrator <span class=""hl"">Databa...",Database_Administrator
2,Oracle Database Administrator Oracle <span cla...,Database_Administrator
3,Amazon Redshift Administrator and ETL Develope...,Database_Administrator
4,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator
...,...,...
29778,"ServiceNow Developer ServiceNow <span class=""h...",Java_Developer\nWeb_Developer\nSoftware_Developer
29779,"Android Developer Android <span class=""hl"">Dev...",Web_Developer\nSoftware_Developer
29780,"UI Developer UI <span class=""hl"">Developer</sp...",Software_Developer\nFront_End_Developer\nWeb_D...
29781,"Web Developer <span class=""hl"">Web</span> <spa...",Web_Developer\nSoftware_Developer


In [15]:
resume_df['lab'] = resume_df['lab'].apply(lambda x: x.replace('\n',' ,'))
resume_df

,txt,lab
0,"Database Administrator <span class=""hl"">Databa...",Database_Administrator
1,"Database Administrator <span class=""hl"">Databa...",Database_Administrator
2,Oracle Database Administrator Oracle <span cla...,Database_Administrator
3,Amazon Redshift Administrator and ETL Develope...,Database_Administrator
4,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator
...,...,...
29778,"ServiceNow Developer ServiceNow <span class=""h...","Java_Developer ,Web_Developer ,Software_Developer"
29779,"Android Developer Android <span class=""hl"">Dev...","Web_Developer ,Software_Developer"
29780,"UI Developer UI <span class=""hl"">Developer</sp...","Software_Developer ,Front_End_Developer ,Web_D..."
29781,"Web Developer <span class=""hl"">Web</span> <spa...","Web_Developer ,Software_Developer"


In [16]:
resume_df['lab'].value_counts()

lab
Systems_Administrator                                                                                2349
Project_manager                                                                                      2339
Database_Administrator                                                                               2225
Software_Developer                                                                                   1991
Web_Developer ,Software_Developer                                                                    1896
                                                                                                     ... 
Systems_Administrator ,Security_Analyst ,Project_manager ,Web_Developer ,Software_Developer             1
Security_Analyst ,Network_Administrator ,Software_Developer ,Systems_Administrator                      1
Network_Administrator ,Web_Developer ,Software_Developer ,Systems_Administrator ,Security_Analyst       1
Security_Analyst ,Systems_Administrator ,P

## Text Cleaning and Embeddings

### Text Cleaning Code

In [20]:
# Load spaCy model with named entity recognition (NER) support
# NER = spacy.load("en_core_web_sm")

def clean_txt(text):
    '''
    function to clean the text
    -- lowercasing the text, removing email ids and website links, stop words and special characters
    -- also it will do lemmitization of the words for better accuray of our model. 
    '''
    # Remove sentences that start with '<span' and end with 'span>'
    cleaned_text = re.sub(r'<span.*?span>', '', text)

    cleaned_text = cleaned_text.lower()# Lowercasing

    cleaned_text = re.sub(r'http\S+|www\S+|https\S+', '', cleaned_text, flags=re.MULTILINE)# Removing hyperlinks

    cleaned_text = re.sub(r'\S+@\S+', '', cleaned_text, flags=re.MULTILINE)# Removing email addresses

    # Removing special characters, numbers, and punctuation
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', cleaned_text)

    # Tokenization
    tokens = word_tokenize(cleaned_text)

    # Removing stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]


    ## below code is to remove city,states and country names. 
    ## not working as expected. need to work on it. 
    # doc = NER(text)
    # Remove entities identified as locations (GPE: Geo-Political Entity)
    # text = ' '.join([token.text for token in doc if token.ent_type_ != 'GPE'])

    return ' '.join(tokens)

In [29]:
# import nltk
# nltk.download()

In [30]:
resume_df['cleaned_txt'] = resume_df['txt'].apply(clean_txt)

In [31]:
resume_df

,txt,lab,cleaned_txt
0,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,database administrator database administrator ...
1,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,database administrator sql server database adm...
2,Oracle Database Administrator Oracle <span cla...,Database_Administrator,oracle database administrator oracle oracle da...
3,Amazon Redshift Administrator and ETL Develope...,Database_Administrator,amazon redshift administrator etl developer bu...
4,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator,scrum master scrum master scrum master richmon...
...,...,...,...
29778,"ServiceNow Developer ServiceNow <span class=""h...","Java_Developer ,Web_Developer ,Software_Developer",servicenow developer servicenow servicenow dev...
29779,"Android Developer Android <span class=""hl"">Dev...","Web_Developer ,Software_Developer",android developer android android developer hc...
29780,"UI Developer UI <span class=""hl"">Developer</sp...","Software_Developer ,Front_End_Developer ,Web_D...",ui developer ui ui developer cotiviti health c...
29781,"Web Developer <span class=""hl"">Web</span> <spa...","Web_Developer ,Software_Developer",web developer los angeles ca work experience w...


### Generating Embeddings

In [96]:
# download below package 
# !python -m spacy download en_core_web_md

In [54]:
GloVe = spacy.load('en_core_web_md')

def embed_resume(text):
    tokens = GloVe(text)
    return np.mean([token.vector for token in tokens], axis=0)


In [58]:
#will take a lot of time so got this bit stackoverflow to do parallel processing 

# Get the number of available CPU cores
num_cores = os.cpu_count()
print(num_cores)

# Use ThreadPoolExecutor for parallel execution
with concurrent.futures.ThreadPoolExecutor(max_workers=num_cores) as executor:
    embeddings_list = list(executor.map(embed_resume, resume_df['txt']))

# Convert the list of embeddings to a NumPy array
embeddings = np.vstack(embeddings_list)

12


In [32]:
embeddings = np.load('embeddings.npy')
embeddings.shape

(29783, 300)

In [67]:
# np.save('embeddings',embeddings)
#saving the embeddings array

## Label Encoding

In [33]:
resume_df['cleaned_lab'] = resume_df['lab'].apply(lambda x: x.split(' ,')[0])
label_encoder = LabelEncoder()
resume_df['labeled_class'] = label_encoder.fit_transform(resume_df['cleaned_lab'])

resume_df

,txt,lab,cleaned_txt,cleaned_lab,labeled_class
0,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,database administrator database administrator ...,Database_Administrator,1
1,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,database administrator sql server database adm...,Database_Administrator,1
2,Oracle Database Administrator Oracle <span cla...,Database_Administrator,oracle database administrator oracle oracle da...,Database_Administrator,1
3,Amazon Redshift Administrator and ETL Develope...,Database_Administrator,amazon redshift administrator etl developer bu...,Database_Administrator,1
4,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator,scrum master scrum master scrum master richmon...,Database_Administrator,1
...,...,...,...,...,...
29778,"ServiceNow Developer ServiceNow <span class=""h...","Java_Developer ,Web_Developer ,Software_Developer",servicenow developer servicenow servicenow dev...,Java_Developer,2
29779,"Android Developer Android <span class=""hl"">Dev...","Web_Developer ,Software_Developer",android developer android android developer hc...,Web_Developer,9
29780,"UI Developer UI <span class=""hl"">Developer</sp...","Software_Developer ,Front_End_Developer ,Web_D...",ui developer ui ui developer cotiviti health c...,Software_Developer,7
29781,"Web Developer <span class=""hl"">Web</span> <spa...","Web_Developer ,Software_Developer",web developer los angeles ca work experience w...,Web_Developer,9


In [34]:
label_encoder.inverse_transform([0,1,2,3,4,5,6,7])

array(['', 'Database_Administrator', 'Java_Developer',
       'Network_Administrator', 'Project_manager', 'Python_Developer',
       'Security_Analyst', 'Software_Developer'], dtype=object)

In [35]:
#dropping those indexes that dont have class names. 
index_drop = resume_df.loc[resume_df['labeled_class'] == 0].index
index_drop

Index([  141,   156,   275,   322,   328,   341,   357,   432,   434,   464,
       ...
       29165, 29198, 29210, 29229, 29257, 29381, 29384, 29386, 29492, 29503],
      dtype='int64', length=748)

In [36]:
resume_df.drop(index_drop,inplace = True)

In [37]:
resume_df['labeled_class'].value_counts()

labeled_class
7    5828
8    4182
4    3527
9    3466
1    2784
2    2418
5    2311
3    2260
6    2259
Name: count, dtype: int64

In [38]:
resume_df.shape

(29035, 5)

In [39]:
embeddings = np.delete(embeddings, index_drop,axis = 0 )
embeddings.shape

(29035, 300)

In [40]:
x_train, x_test, y_train, y_test =train_test_split(embeddings,resume_df['labeled_class'],test_size= 0.2, random_state= 69)

In [41]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

x_train (23228, 300)
x_test (5807, 300)
y_train (23228,)
y_test (5807,)


In [111]:
# resume_df.to_csv('resume_df', index = False)
# Saving resume_df locally

## Model Building

### Checking and Enabling GPU for Tensorflow

In [1]:
import tensorflow as tf

In [44]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
import sys

import tensorflow.keras
# import pandas as pd
# import sklearn as sk
import tensorflow as tf

# print(f"Tensor Flow Version: {tf.__version__}")
# print(f"Keras Version: {tensorflow.keras.__version__}")
print()
# print(f"Python {sys.version}")
# print(f"Pandas {pd.__version__}")
# print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")


GPU is available


In [48]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [46]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [108]:
resume_df

,txt,lab,cleaned_txt,cleaned_lab,labeled_class
0,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,database administrator database administrator ...,Database_Administrator,1
1,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,database administrator sql server database adm...,Database_Administrator,1
2,Oracle Database Administrator Oracle <span cla...,Database_Administrator,oracle database administrator oracle oracle da...,Database_Administrator,1
3,Amazon Redshift Administrator and ETL Develope...,Database_Administrator,amazon redshift administrator etl developer bu...,Database_Administrator,1
4,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator,scrum master scrum master scrum master richmon...,Database_Administrator,1
...,...,...,...,...,...
29778,"ServiceNow Developer ServiceNow <span class=""h...","Java_Developer ,Web_Developer ,Software_Developer",servicenow developer servicenow servicenow dev...,Java_Developer,2
29779,"Android Developer Android <span class=""hl"">Dev...","Web_Developer ,Software_Developer",android developer android android developer hc...,Web_Developer,9
29780,"UI Developer UI <span class=""hl"">Developer</sp...","Software_Developer ,Front_End_Developer ,Web_D...",ui developer ui ui developer cotiviti health c...,Software_Developer,7
29781,"Web Developer <span class=""hl"">Web</span> <spa...","Web_Developer ,Software_Developer",web developer los angeles ca work experience w...,Web_Developer,9


### Training Model

In [105]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input
import warnings 
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [106]:
tf.debugging.set_log_device_placement(True)
# tf.get_logger().setLevel('FATAL')

model = Sequential()
model.add(Input(shape=(300,1)))
model.add(SimpleRNN(64, activation='relu'))
model.add(Dense(9, activation='softmax'))  

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing o

2023-11-30 13:12:33.955943: I tensorflow/core/common_runtime/placer.cc:125] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-11-30 13:12:33.955957: I tensorflow/core/common_runtime/placer.cc:125] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2023-11-30 13:12:33.966887: I tensorflow/core/common_runtime/placer.cc:125] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-11-30 13:12:33.966900: I tensorflow/core/common_runtime/placer.cc:125] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2023-11-30 13:12:33.971286: I tensorflow/core/common_runtime/placer.cc:125] resource_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2023-11-30 13:12:33.971297: I tensorflow/core/common_runtime/placer.cc:125] VarHandleOp: (VarHandleOp): /job:localhost/replica:0/task:0/device:GPU:0
2023-11-30 13:12:33.991694: I tensorflow/core/common_runtime/placer.cc:125] resource_RetVal: (_Retval): /j

In [107]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_16 (SimpleRNN)   (None, 64)                4224      
                                                                 
 dense_11 (Dense)            (None, 9)                 585       
                                                                 
Total params: 4809 (18.79 KB)
Trainable params: 4809 (18.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model (adjust loss and metrics based on your task)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, np.array(y_train).reshape(-1,1), epochs=10, batch_size=32, validation_split=0.1)

In [ ]:
x_train[0].reshape(-1,1)